In [2]:
from grids import *
import os

In [3]:
#real-space centroid of the box (in this case, centroid of BU72 in PDB: 5C1M)
grid_center = "0.33, 1.64, 9."

#docking precision
precision = "SP"

#directories
base_dir = "/home/enf/md_simulations/cb1"
ligands_dir = "%s/ligands" %base_dir #replace with where we copied the mol files
proteins_dir = "%s/proteins" %base_dir #replace with where we copied the protein conformers 
grids_dir = "%s/grids" %base_dir #replace ~ with the base directory you're using for the tutorial

if not os.path.exists(base_dir):
    os.makedirs(base_dir)
if not os.path.exists(ligands_dir):
    os.makedirs(ligands_dir)
if not os.path.exists(proteins_dir):
    os.makedirs(proteins_dir)
if not os.path.exists(grids_dir):
    os.makedirs(grids_dir)

docking_dir = "%s/docking_%s" %(base_dir, precision) #replace BASE_DIR with some directory 
if not os.path.exists(docking_dir):
    os.makedirs(docking_dir)
    
#protein structure to which we align
active_ref_dir = "/home/enf/md_simulations/MOR/bu72_ref_receptors/conformation/bu72_ionized_pymol_RL_conformation.pdb" #copy this file from vsp-compute


In [5]:
#download cannabinoid compounds
cannabinoid_cids = [2543,
                    10382701,
                    5311501,
                    6918505,
                    9821569,
                    107778,
                    3083542,
                    4302963,
                    644019,
                    5712057,
                    3081355,
                    5361881,
                    53393997,
                    16078,
                    10471670]
download_sdfs_from_cids(cannabinoid_cids, ligands_dir, worker_pool=None, parallel=True)

In [6]:

#protein preparation
pprep(proteins_dir, ref = active_ref_dir, chosen_receptors = None, worker_pool=None, parallel=False)

#generate docking grids
generate_grids(proteins_dir, grid_center,
               grids_dir, remove_lig = "LIG",
               chosen_receptors = None,
               worker_pool=None, outer_box=25.,
               parallel=False)

#prepare ligands for docking
ligand_df = prepare_ligands(ligands_dir, exts = [".mol", ".sdf"],
                            n_ring_conf=6, n_stereoisomers=32,
                            force_field=16, worker_pool=None,
                            parallel=True, redo=False,
                            smiles_df=None)

chosen_receptors=None

1
['/home/enf/md_simulations/cb1/proteins/5tgz.pdb']
$SCHRODINGER/utilities/prepwizard -WAIT -disulfides -fix -noepik -noimpref -noprotassign -reference_st_file /home/enf/md_simulations/MOR/bu72_ref_receptors/conformation/bu72_ionized_pymol_RL_conformation.pdb -NOLOCAL /home/enf/md_simulations/cb1/proteins/5tgz.pdb 5tgz.mae
/vspdata/vspcompute/home/enf/md_simulations/cb1/proteins
Done prepping proteins
/home/enf/md_simulations/cb1/grids
cp /home/enf/md_simulations/cb1/proteins/5tgz.mae /home/enf/md_simulations/cb1/grids/5tgz.mae
$SCHRODINGER/glide /home/enf/md_simulations/cb1/grids/5tgz.in -OVERWRITE -WAIT
completed grid generation job
Examining 15 ligands
finished preparing ligands


NameError: name 'dview' is not defined

In [7]:
#dock

dock_ligands_and_receptors(grids_dir, docking_dir, ligands_dir,
                           precision = precision, ext = "-out.maegz",
                           parallel = True,
                           grid_ext = ".zip", worker_pool=None, retry_after_failed=False,                           
                           timeout=60*60)

Creating new directories for each ligand.
Done creating directories. Determining which docking jobs to conduct.
12
0.18509483337402344
15
About to do 15 Docking computations.
timeout 3600 $SCHRODINGER/glide /home/enf/md_simulations/cb1/docking_SP/CID_107778/5tgz.in -OVERWRITE -WAIT -strict -NOJOBID > 5tgz.log
timeout 3600 $SCHRODINGER/glide /home/enf/md_simulations/cb1/docking_SP/CID_10382701/5tgz.in -OVERWRITE -WAIT -strict -NOJOBID > 5tgz.log
timeout 3600 $SCHRODINGER/glide /home/enf/md_simulations/cb1/docking_SP/CID_10471670/5tgz.in -OVERWRITE -WAIT -strict -NOJOBID > 5tgz.log
timeout 3600 $SCHRODINGER/glide /home/enf/md_simulations/cb1/docking_SP/CID_16078/5tgz.in -OVERWRITE -WAIT -strict -NOJOBID > 5tgz.log
timeout 3600 $SCHRODINGER/glide /home/enf/md_simulations/cb1/docking_SP/CID_2543/5tgz.in -OVERWRITE -WAIT -strict -NOJOBID > 5tgz.log
timeout 3600 $SCHRODINGER/glide /home/enf/md_simulations/cb1/docking_SP/CID_3081355/5tgz.in -OVERWRITE -WAIT -strict -NOJOBID > 5tgz.log
timeout

In [16]:
docking_dir

'/home/enf/md_simulations/MOR/h8_reimaged/sparse-tICA_t50_n_components2all_residues_4dkl_5c1m_under_cutoff6A-CA-py3-far_2_regularization_wolf_autoShrinkage0pt01-backup/all_clusterer_25clusters_1samples_samples_kdtree/docking_SP_32-stereoisomers_6-ring-conf/htbc'

In [16]:

docking_df, poses_df = analyze_docking_results_multiple(docking_dir, precision, "%s/summary.pkl" %docking_dir,
                                                        poses_summary=None, redo=False, reread=True,
                                                        write_to_disk=True, worker_pool=None, parallel=True)


